In [24]:
import os
import io
import struct
from typing import List, Tuple, Optional
from enum import Enum
import pandas as pd

from blowfish import Blowfish

In [25]:
class PackEntryType(Enum):
  Nop = 0
  Folder = 1
  File = 2

In [26]:
def generate_final_blowfish_key(password: str, salt: bytes) -> bytes:
    """
    Reproduces the 'GenerateFinalBlowfishKey' logic in C#.
    """
    # 1) Limit key length to max of 56
    plain_key_length = min(len(password), 56)

    # 2) Convert password to ASCII bytes
    a_key = password.encode("ascii")

    # 3) Create a 56-byte base key buffer
    b_key = bytearray(56)
    
    # Copy salt into b_key
    # Equivalent to `Array.ConstrainedCopy(salt, 0, bKey, 0, salt.Length)`
    b_key[:len(salt)] = salt

    # 4) Generate the final blowfish key by XOR-ing
    #    the ASCII password bytes with the corresponding part of b_key.
    bf_key = bytearray(plain_key_length)
    for x in range(plain_key_length):
        bf_key[x] = a_key[x] ^ b_key[x]

    return bytes(bf_key)

In [27]:
def read_string_with_length(stream, byte_count: int) -> str:
    """
    Read 'byte_count' bytes, then decode using self._encoding,
    stopping at the first 0 (null terminator) if present.
    """
    buffer = stream.read(byte_count)

    # Find null terminator offset (if any)
    terminator_offset = byte_count
    for i in range(byte_count):
        if buffer[i] == 0:
            terminator_offset = i
            break

    return buffer[:terminator_offset].decode('ascii', errors='replace')

In [28]:
# data_pk2_path = os.path.join(os.getcwd(), 'data', 'Data.pk2')
media_pk2_path = os.path.join(os.getcwd(), 'data', 'Media.pk2')
# print(data_pk2_path)
print(media_pk2_path)

/Users/hodung/Desktop/Workspace/silkroad/pk2-extractor/LoadTextData/data/Media.pk2


In [29]:
pk2key = '169841'
salt = [0x03, 0xF8, 0xE4, 0x44, 0x88, 0x99, 0x3F, 0x64, 0xFE, 0x35]

In [30]:
key = generate_final_blowfish_key(pk2key, bytes(salt))
key

b'2\xce\xdd|\xbc\xa8'

In [31]:
blowfish = Blowfish()
blowfish.Initialize(key)

In [32]:
_media_file_stream = open(media_pk2_path, 'rb')

In [33]:
signature = _media_file_stream.read(30)
version =  struct.unpack('<i', _media_file_stream.read(4))[0]
encrypted = _media_file_stream.read(1)
encryption_checksum = _media_file_stream.read(16)
payload = _media_file_stream.read(205)


In [34]:
Header = {
    'signature': signature,
    'version': version,
    'encrypted': encrypted,
    'encryption_checksum': encryption_checksum,
    'payload': payload
}

In [35]:
blowfish_checksum_decoded = "Joymax Pak File"

In [36]:
if blowfish and encrypted == b'\x01':
    temp_checksum = blowfish.Encode(blowfish_checksum_decoded.encode('ascii'))
    if temp_checksum is None or temp_checksum[0] != encryption_checksum[0] or temp_checksum[1] != encryption_checksum[1] or temp_checksum[2] != encryption_checksum[2]:
        raise Exception('Failed to open JoymaxPackFile: The password or salt is wrong.')

In [37]:
def read_block_at(position: int):
    _media_file_stream.seek(position, io.SEEK_SET)
    buffer = _media_file_stream.read(128 * 20)
    if blowfish is not None:
      entry_buffer = io.BytesIO(blowfish.Decode(buffer))
    else:
      entry_buffer = io.BytesIO(buffer)

    entries = []
    for _ in range(20):
      entry = {
        "Type": PackEntryType(entry_buffer.read(1)[0]),
        "Name": read_string_with_length(entry_buffer, 89).rstrip('\0'),
        "CreationTime": struct.unpack('<q', entry_buffer.read(8))[0],
        "ModifyTime": struct.unpack('<q', entry_buffer.read(8))[0],
        "DataPosition": struct.unpack('<q', entry_buffer.read(8))[0],
        "DataSize": struct.unpack('<i', entry_buffer.read(4))[0],
        "NextBlock": struct.unpack('<q', entry_buffer.read(8))[0],
        "Payload": entry_buffer.read(2)
      }
      entries.append(entry)
      # print(entry)

    return {
      "Position": position,
      "Entries": entries
    }


def read_blocks_at(position: int):
    result = []

    block = read_block_at(position)
    result.append(block)

    if block["Entries"][19]["NextBlock"] > 0:
        result.extend(read_blocks_at(block["Entries"][19]["NextBlock"]))

    return result

In [38]:
Root = read_blocks_at(256)

In [39]:
blocks_in_memory = {
  "": Root
}

In [40]:
blocks_in_memory

{'': [{'Position': 256,
   'Entries': [{'Type': <PackEntryType.Folder: 1>,
     'Name': '.',
     'CreationTime': 133336205048819447,
     'ModifyTime': 133336205048819447,
     'DataPosition': 256,
     'DataSize': 0,
     'NextBlock': 0,
     'Payload': b'\x00\x00'},
    {'Type': <PackEntryType.Folder: 1>,
     'Name': 'acobject',
     'CreationTime': 133336205048829450,
     'ModifyTime': 133336205048829450,
     'DataPosition': 2816,
     'DataSize': 0,
     'NextBlock': 0,
     'Payload': b'\x00\x00'},
    {'Type': <PackEntryType.Folder: 1>,
     'Name': 'config',
     'CreationTime': 133336205049179438,
     'ModifyTime': 133336205049179438,
     'DataPosition': 35136,
     'DataSize': 0,
     'NextBlock': 0,
     'Payload': b'\x00\x00'},
    {'Type': <PackEntryType.Folder: 1>,
     'Name': 'Effect',
     'CreationTime': 133336205049679437,
     'ModifyTime': 133336205049679437,
     'DataPosition': 41896,
     'DataSize': 0,
     'NextBlock': 0,
     'Payload': b'\x00\x00'},
   

In [41]:
def get_entry_buffer(file_path: str) -> Optional[dict]:
  parent_folder_path = os.path.dirname(file_path)
  file_name = os.path.basename(file_path)
  if parent_folder_path not in blocks_in_memory:
    paths = parent_folder_path.split(os.path.sep)
    blocks = blocks_in_memory[""]
    current_path = ""

    for sub_folder_name in paths:
        for block in blocks:
            entries = block["Entries"]
            entry = next((e for e in entries if e["Name"] == sub_folder_name and e["Type"] == PackEntryType.Folder), None)
            if entry is None:
                continue
            
            current_path = os.path.join(current_path, entry["Name"])
            if current_path in blocks_in_memory:
                blocks = blocks_in_memory[current_path]
                break
            
            blocks = read_blocks_at(entry["DataPosition"])
            blocks_in_memory[current_path] = blocks
            break
        
  root = blocks_in_memory[parent_folder_path]
  entry = None
  for entries in root:
    entry = next((x for x in entries["Entries"] if x["Name"] == file_name.lower()), None)
    if entry:
      break

  _media_file_stream.seek(entry["DataPosition"], io.SEEK_SET)
  # _media_file_stream.read(entry["DataSize"])
  buffer = io.BytesIO(_media_file_stream.read(entry["DataSize"])) 

  return buffer
   

def get_lines(file_path: str) -> List[Tuple[str, int]]:

  buffer = get_entry_buffer(file_path)

  text = buffer.read().decode('utf-16', errors="replace")
  lines = text.split('\r\n')
  return lines  

def convert_string(value):
    try:
        # First, try to convert to an integer
        return int(value)
    except ValueError:
        try:
            # If it fails, try to convert to a float
            return float(value)
        except ValueError:
            # If both conversions fail, return the original string
            return value


In [42]:
text_data_directory = os.path.join('server_dep', 'silkroad', 'textdata')
text_data_directory

'server_dep/silkroad/textdata'

## Load Text Data

### TextQuest_Speech&Name.txt

In [43]:
text_quest_quest_string_list_file = os.path.join(text_data_directory, 'TextQuest_QuestString.txt')
text_quest_quest_string_list_file

'server_dep/silkroad/textdata/TextQuest_QuestString.txt'

In [44]:
text_quest_quest_string_files = get_lines(text_quest_quest_string_list_file)
text_quest_quest_string_files

['TextQuest_QuestString_100.txt',
 'TextQuest_QuestString_1000.txt',
 'TextQuest_QuestString_10000.txt',
 'TextQuest_QuestString_10050.txt',
 'TextQuest_QuestString_10100.txt',
 'TextQuest_QuestString_10150.txt',
 'TextQuest_QuestString_10200.txt',
 'TextQuest_QuestString_10250.txt',
 'TextQuest_QuestString_10300.txt',
 'TextQuest_QuestString_10350.txt',
 'TextQuest_QuestString_10400.txt',
 'TextQuest_QuestString_10450.txt',
 'TextQuest_QuestString_1050.txt',
 'TextQuest_QuestString_10500.txt',
 'TextQuest_QuestString_10550.txt',
 'TextQuest_QuestString_10600.txt',
 'TextQuest_QuestString_10650.txt',
 'TextQuest_QuestString_10700.txt',
 'TextQuest_QuestString_10750.txt',
 'TextQuest_QuestString_10800.txt',
 'TextQuest_QuestString_10850.txt',
 'TextQuest_QuestString_10900.txt',
 'TextQuest_QuestString_10950.txt',
 'TextQuest_QuestString_1100.txt',
 'TextQuest_QuestString_11000.txt',
 'TextQuest_QuestString_11050.txt',
 'TextQuest_QuestString_11100.txt',
 'TextQuest_QuestString_11150.txt

In [45]:
df_text_quest_quest_string = pd.DataFrame(columns=range(17))

for text_quest_quest_string_file in text_quest_quest_string_files:
  file_name = os.path.join(text_data_directory, text_quest_quest_string_file)
  lines = get_lines(file_name)
  for idx, line in enumerate(lines):
    values = line.split('\t')
    if len(values) < 17:
      continue
    df_text_quest_quest_string.loc[len(df_text_quest_quest_string)] = values
    


In [46]:
df_text_quest_quest_string

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,1,50,SN_TALK_QTUTORIAL_CH_36,노점을 여는 법을 알려주겠네.,0,0,0,0,0,I'll explain how to open a stall.,Tôi sẽ giải thích cách mở một gian hàng riêng,0,0,0,0,0,0
1,1,51,SN_TALK_QTUTORIAL_CH_37,사냥을 하다 보면 여러 가지 물건들을 습득할 수 있는데 그것들을 상점에 팔면 물건의...,0,0,0,0,0,"As you hunt, you will obtain various items. If...","Khi đi săn, bạn sẽ tình cơ có được rất nhiều đ...",0,0,0,0,0,0
2,1,52,SN_TALK_QTUTORIAL_CH_38,이것은 이 험한 세상을 헤쳐나가는데 조금이나마 도움을 줄 수 있을까 해서 주는 것이...,0,0,0,0,0,Please take this. I give you this hoping it wi...,Hãy cầm lấy cái này. Mong rằng nó có ích cho b...,0,0,0,0,0,0
3,1,53,SN_TALK_QTUTORIAL_CH_39,기본행동 법을 알려주겠네.\n오른쪽 하단의 메뉴버튼을 누르면 등장하는 창에서 왼쪽에...,0,0,0,0,0,I'll teach you the basic game actions.\nClick ...,Tôi sẽ chỉ cho bạn những hành động cơ bản.\nBạ...,0,0,0,0,0,0
4,1,54,SN_TALK_QTUTORIAL_CH_40,아이템의 사용에 대해 알려주겠네.\n메뉴 버튼을 눌러 왼쪽에서 두 번째 버튼을 누르...,0,0,0,0,0,I'll explain how to use items.\nPress the menu...,"Bây giờ, tôi sẽ giải thích cho bạn cách sử dụn...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12464,1,9945,SN_TALK_QNO_N_KT_082_02,"페논 파이터 308마리, 페논 워리어 330마리를 쓰러뜨리게. 호탄 왕국을 지키기 ...",0,0,0,0,0,Slay 308 Penon Fighters and 330 Penon Warriors...,Giết 308 Lam Băng Binh và 330 Khôi Băng Binh.,0,0,0,0,0,0
12465,1,9946,SN_TALK_QNO_N_KT_082_03,자네가 상대하기엔 너무 강한 상대들인가?,0,0,0,0,0,Are they too powerful for you?,Chúng quá mạnh mẽ so với bạn?,0,0,0,0,0,0
12466,1,9947,SN_TALK_QNO_N_KT_082_04,"페논 파이터와 페논 워리어를 물리치게나. 각각 308마리, 330마리를 처치하면 된다네.",0,0,0,0,0,Slay Penon Fighters and Penon Warriors. Slay ...,Giết 308 Lam Băng Binh và 330 Khôi Băng Binh.,0,0,0,0,0,0
12467,1,9948,SN_TALK_QNO_N_KT_082_05,쉽지만은 않은 상대였을텐데 잘 처리하였군. 하지만 방심해서는 안 되네. 언제 다시 ...,0,0,0,0,0,You have done so well against such tough foes....,Bạn đã làm rất tốt. Nhưng bạn không được phép ...,0,0,0,0,0,0
